In [1]:
%run env_setup.py
%matplotlib notebook

In [2]:
from matplotlib import pyplot as plt
#from tensorflow.contrib import keras
import keras
import bcolz
from lessdeep.utils import download_file, extract_file
from lessdeep.model.vgg16n import Vgg16N

D:\dev\libs\Anaconda3\envs\tf140p35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Super resolution

In [3]:
resized_72_path = extract_file(download_file('http://files.fast.ai/data/trn_resized_72.tar.gz'))
resized_288_path = extract_file(download_file('http://files.fast.ai/data/trn_resized_288.tar.gz'))

In [4]:
arr_lr = bcolz.open(resized_72_path) # Low resolution
arr_hr = bcolz.open(resized_288_path) # High resolution

In [5]:
def conv_block(x, filters, size, strides=(2,2), padding='same', activation=keras.activations.relu):
    x = keras.layers.Conv2D(filters, kernel_size=size, strides=strides, padding=padding)(x)
    x = keras.layers.BatchNormalization()(x)
    return activation(x) if activation else x

def res_block(x, filters=64):
    y = conv_block(x, filters, 3, strides=(1,1))
    y = conv_block(y, filters, 3, strides=(1,1), activation=None)
    return keras.backend.sum([y, x], axis=0)

def up_block(x, filters, size):
    x = keras.layers.UpSampling2D()(x)
    x = keras.layers.Conv2D(filters, size, padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    return keras.activations.relu(x)

def deconv_block(x, filters, size, strides=(2,2)):
    x = keras.layers.Conv2DTranspose(filters, size, strides=strides, padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    return keras.activations.relu(x)

In [6]:
inp = keras.layers.Input(arr_lr.shape[1:])
x = conv_block(inp, 64, 9, strides=(1,1))
for _ in range(4):
    x = res_block(x)
x = up_block(x, 64, 3)
x = up_block(x, 64, 3)
x = keras.layers.Conv2D(3, 9, activation='tanh', padding='same')(x)
out = keras.layers.Lambda(lambda x: (x + 1)*127.5)(x)

In [7]:
vgg = Vgg16N(include_top=False, image_size=arr_hr.shape[1:3])
vgg_inp = vgg.model.input

In [8]:
for l in vgg.model.layers: l.trainable=False

In [9]:
def get_layer_out(model, i):
    return model.model.layers[i].output

conv_layers = keras.models.Model(vgg_inp, [get_layer_out(vgg, i) for i in [1, 4, 7]])
vgg1 = conv_layers(out)
vgg2 = conv_layers(vgg_inp)

<tf.Tensor 'input_4:0' shape=(?, 288, 288, 3) dtype=float32>

In [50]:
keras.__file__

'D:\\dev\\libs\\Anaconda3\\envs\\tf140p35\\lib\\site-packages\\tensorflow\\contrib\\keras\\__init__.py'

In [10]:
def image_compare(img):
    fig, (ax1, ax2) = plt.subplots(1,2)
    ax1.imshow(img)
    ax2.imshow(img)
    plt.show()